# Đề xuất Chiến thuật Bóng đá bằng Thuật toán Gaussian Naive Bayes

## 1. Giới thiệu
Notebook này thực hiện quy trình xây dựng mô hình học máy để:
1. Dự đoán kết quả trận đấu dựa trên thông số kỹ thuật.
2. Phân tích đặc trưng của các đội chiến thắng.
3. Đề xuất điều chỉnh chiến thuật.

**Lưu ý quan trọng:** Quy trình này tuân thủ nghiêm ngặt việc chia tập dữ liệu trước khi chuẩn hóa để tránh rò rỉ dữ liệu (Data Leakage).

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')

## 2. Tải và Khám phá Dữ liệu

In [ ]:
# Tải dữ liệu
df = pd.read_csv('tactical_dataset_with_results.csv')

# Lựa chọn các biến chiến thuật (Features)
features = ['possession_ratio', 'num_passes', 'avg_pass_length', 'shots', 'xg', 
            'pressures', 'tackles', 'interceptions', 'avg_x_position', 
            'team_width', 'final_third_actions', 'ppda']
target = 'result'

# Kiểm tra phân phối nhãn
print("Phân phối kết quả trận đấu:")
print(df[target].value_counts(normalize=True))

## 3. Chia tách Dữ liệu (Data Splitting)
**Quan trọng:** Sử dụng `stratify=y` để đảm bảo tỷ lệ Thắng/Thua/Hòa trong tập Train và Test là giống nhau.

In [ ]:
X = df[features]
y = df[target]

# Chia tập dữ liệu 80% Train - 20% Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Kích thước tập Train: {X_train.shape}")
print(f"Kích thước tập Test: {X_test.shape}")

## 4. Tiền xử lý (Preprocessing)
Chuẩn hóa dữ liệu (Standard Scaling) để đưa các biến về cùng một thang đo (mean=0, std=1). 
**Lưu ý:** Chỉ `fit` trên tập Train, sau đó `transform` trên tập Test. Tuyệt đối không fit trên toàn bộ dữ liệu.

In [ ]:
scaler = StandardScaler()

# Fit trên train và transform train
X_train_scaled = scaler.fit_transform(X_train)

# Chỉ transform trên test (sử dụng tham số từ train)
X_test_scaled = scaler.transform(X_test)

# Chuyển đổi lại thành DataFrame để dễ quan sát (tùy chọn)
X_train_df = pd.DataFrame(X_train_scaled, columns=features)
X_test_df = pd.DataFrame(X_test_scaled, columns=features)

## 5. Huấn luyện Mô hình Gaussian Naive Bayes

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train_scaled, y_train)

print("Đã huấn luyện xong mô hình.")

## 6. Đánh giá Mô hình

In [ ]:
y_pred = gnb.predict(X_test_scaled)

print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Vẽ Confusion Matrix
plt.figure(figsize=(8,6))
cm = confusion_matrix(y_test, y_pred, labels=gnb.classes_)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=gnb.classes_, yticklabels=gnb.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

## 7. Trích xuất "DNA Chiến thắng" (Tactical Blueprint)
Phần này thực hiện mục tiêu "Đề xuất": Tìm ra các chỉ số trung bình của nhóm thắng cuộc.

In [ ]:
# Tìm index của class 'Win'
win_idx = np.where(gnb.classes_ == 'Win')[0][0]

# Lấy giá trị trung bình (mean) của các đặc trưng cho class Win (đang ở dạng scaled)
win_means_scaled = gnb.theta_[win_idx]

# Chuyển ngược lại về giá trị thực tế
win_means_real = scaler.inverse_transform([win_means_scaled])[0]

# Tạo bảng so sánh
blueprint_df = pd.DataFrame({'Feature': features, 'Target_Win_Value': win_means_real})
blueprint_df = blueprint_df.sort_values(by='Target_Win_Value', ascending=False)
print(blueprint_df)

## 8. Hệ thống Đề xuất Chiến thuật (Tactical Recommender)
Hàm dưới đây so sánh chỉ số hiện tại của một đội với "Target Win Value" để đưa ra lời khuyên.

In [ ]:
def propose_tactics(current_stats, model, scaler, features):
    # 1. Dự đoán kết quả hiện tại
    current_scaled = scaler.transform([current_stats])
    prediction = model.predict(current_scaled)[0]
    probs = model.predict_proba(current_scaled)[0]
    
    print(f"Dự đoán hiện tại: {prediction}")
    print(f"Xác suất: Win: {probs[2]:.2f}, Draw: {probs[0]:.2f}, Loss: {probs[1]:.2f}")
    
    if prediction == 'Win':
        print("==> Đội đang đi đúng hướng!")
        return
    
    # 2. Đề xuất cải thiện
    win_idx = np.where(model.classes_ == 'Win')[0][0]
    win_means = scaler.inverse_transform([model.theta_[win_idx]])[0]
    
    print("\n--- ĐỀ XUẤT ĐIỀU CHỈNH ---")
    suggestions = []
    for i, feature in enumerate(features):
        diff = win_means[i] - current_stats[i]
        if abs(diff) > win_means[i] * 0.1: # Chỉ đề xuất nếu lệch > 10%
            direction = "Tăng" if diff > 0 else "Giảm"
            suggestions.append((feature, direction, abs(diff)))
            
    # Sắp xếp theo độ lệch giảm dần
    suggestions.sort(key=lambda x: x[2], reverse=True)
    
    for item in suggestions[:3]: # Top 3 đề xuất
        print(f"- Cần {item[1]} {item[0]} thêm khoảng {item[2]:.2f} đơn vị")

# Test với một mẫu ngẫu nhiên từ tập Test (giả sử là mẫu thất bại)
sample_idx = 0 
sample_stats = X_test.iloc[sample_idx].values
print("Stats mẫu:", dict(zip(features, sample_stats)))
propose_tactics(sample_stats, gnb, scaler, features)